In [1]:
%pylab notebook

Populating the interactive namespace from numpy and matplotlib


# Random walk

Hamiltonian (from lecture):

$\mathcal{H}=-\frac{1}{2}\sum_{i,j}J_{\text{ij}}S_{\text{i}}S_{\text{j}}-B\sum_{i}S_{\text{i}}$

Hamiltonian (general):

$\mathcal{H}=-\frac{1}{2}\sum_{i=1}^{N}\sum_{j=1;j \neq i}^{N}J_{\text{ij}}S_{\text{i}}S_{\text{j}}-B\sum_{i}S_{\text{i}}$

Hamiltonian (pairs of next neighbor interaction and equal interaction strength $J_{\text{ij}}=J$ ):

$\mathcal{H}=-J\sum_{<i,j>}^{N,N}S_{\text{i}}S_{\text{j}}-B\sum_{i}S_{\text{i}}$

Simple sampling observables:

$\langle Q \rangle = \frac{\sum_i Q_i \exp{(-\beta E_i)}}{\sum_i\exp{(-\beta E_i)}}$ mit $\beta = 1/k_\text{B}T$

In [2]:
class randomWalker:
    def __init__(self,L_ = 20, J_ = 1, B_ = 0):
        self.L = L_
        self.J = J_
        self.B = B_
        self.lattice = np.zeros((L_,L_),dtype=np.int)
        
    def randomize(self):
        ''' create new random array with -1 or 1 ''' 
        self.lattice = (2*np.random.randint(2,size = self.lattice.shape))-1
    #

    def plotConfig(self):
        ''' plot the configs using a simple colormesh plot '''
        figure()
        pcolormesh(self.lattice)
        xlabel('Y')
        ylabel('X')
        title('Ising spin lattice')
        show()
    #
    
    def getMagnetisation(self):
        return float(self.lattice.sum())/self.lattice.size
    
    # care for periodic boundaries
    def foldBack(self, idx):
        if (idx < 0):
            return (self.L-1)
        elif (idx == self.L):
            return 0
        else:
            return idx
    
    ''' get single spin energy to be prepared for metropolis '''
    def getSingleSpinEnergy(self, x, y):
        sumSpins = self.lattice[x,self.foldBack(y+1)] + self.lattice[x,self.foldBack(y-1)] + self.lattice[self.foldBack(x+1),y] + self.lattice[self.foldBack(x-1),y]
        return -1.0 * self.lattice[x,y] * (0.5 * self.J * sumSpins + self.B)
        
    ''' get total energy of the full lattice '''
    def getEnergy(self):
        E = 0
        for x in range(self.L):
            for y in range(self.L):
                E += self.getSingleSpinEnergy(x,y)
        # normalize total energy between -1 and +1
        return E/(2.0*self.lattice.size)
    
    ''' perform Metropolis move ''' 
    def performMCS(self, temperature, time):
        # introduce counter to check for frozen states
        counter = 0
        
        # check if lattice was initialised
        if self.lattice[0,0] == 0:
            self.randomize()
        
        # loop over number of timesteps
        for t in range(time):
            for i in range(self.L*self.L):
                # get random positions
                x = np.random.randint(0,self.L)
                y = np.random.randint(0,self.L)
                
                # get actuall spin energy
                e_old = self.getSingleSpinEnergy(x,y)
                # new energy is just with flipped spin -> -1
                # e_new = -e_old -> Delta E = 2 e_old
                # calculate probability from energy difference
                prob = np.exp( - 2* e_old / temperature )
                
                # apply metropolis 
                if (e_old < 0):
                    self.lattice[x,y] *= -1
                    counter += 1
                elif (np.random.random_sample() < prob):
                    self.lattice[x,y] *= -1
                    counter += 1
        return counter
        

In [3]:
# make some test output
hugo = randomWalker(20)
hugo.randomize()
#hugo.plotConfig()
hugo.lattice[1,0]=1
hugo.lattice[0,3]=1
hugo.plotConfig()
print("size = ",hugo.L, "magnetisation = ", hugo.getMagnetisation())

print("spin energy of 0,0 = ",hugo.getSingleSpinEnergy(0,0))
print("total energy = ",hugo.getEnergy())


<IPython.core.display.Javascript object>

size =  20 magnetisation =  0.06
spin energy of 0,0 =  -0.0
total energy =  0.05


In [4]:
# run the simulations of the simple sampling part
MyWalker = randomWalker(10)
# in beta =1/T ...
#temp = np.array([0.2,0.25,1/3,1/2.5,1/2.27,1/2.2,1/2.2,0.5,1,1.5,2,5,10])
temp = np.array([5,4,3,2.5,2.27,2.2,2.1,2,1.5,1,0.5,0.2])

meanM = np.zeros(temp.size)
meanE = np.zeros(temp.size)
normalization = np.zeros(temp.size)
hugo = randomWalker(20)
for n in range(10000):
    # get a new config
    MyWalker.randomize()
    # calculate unweighted magnetisation and energy
    E = MyWalker.getEnergy()
    M = MyWalker.getMagnetisation()
    # calculate boltzmann factor
    expE = np.exp(-E/temp)
    # add weighted observables to means
    meanM += M*expE
    meanE += E*expE
    normalization += expE
    
print(meanM/normalization,"\n",meanE/normalization)

[-0.00201107 -0.00200324 -0.00199007 -0.00197945 -0.00197296 -0.0019707
 -0.00196722 -0.00196337 -0.00193612 -0.00187982 -0.00169287 -0.00077883] 
 [-0.00269086 -0.00369103 -0.00535757 -0.00669043 -0.00750058 -0.00778075
 -0.00821336 -0.0086892  -0.01201926 -0.01867628 -0.03865419 -0.09965701]


In [5]:
figure()

# subplot magnetisation
subplot(211)
plot(temp,meanM/normalization, label="magnetisation")
#plot(tempertatures,meanE/normalization, label="energy")
xlabel("T")
ylabel("M")
xlim(0.1,5)
legend(loc="best")
title("Ising Magnetisation Simple Sampling")

# subplot energy
subplot(212)
plot(temp,meanE/normalization, label="energy")
#plot(tempertatures,meanE/normalization, label="energy")
xlabel("T")
ylabel("E")
xlim(0.1,5)
legend(loc="best")
title("Ising Energy Simple Sampling")

# adjust plot spacing
subplots_adjust(hspace=.5)

show()

<IPython.core.display.Javascript object>

In [6]:
# define class instance
TheWalker = randomWalker(20)

# setup parameters
n_samples = 100
n_timesteps = 200
#temp = np.array([5,4,3,2.5,2.27,2.2,2.1,2,1.5,1,0.5,0.2])
myTemp = 0.1

# setup data container
data = np.zeros((n_samples,n_timesteps))

# run sime resolved simulations
for i in range(n_samples):
    TheWalker.randomize()
    for t in range(n_timesteps):
        check = TheWalker.performMCS(0.1,1)
        #if (check):
        #    print("nothing moved")
        # use direct sampling without boltzmann weigth
        data[i,t] = TheWalker.getMagnetisation()
    # - if there are strange values, check some final configs for a few n_samples
    #TheWalker.plotConfig()
#

KeyboardInterrupt: 

In [ ]:
figure()
x = np.linspace(1,n_timesteps,n_timesteps)
plot(x,data[0,:], label= "run 1")
plot(x,data[1,:], label= "run 2")
plot(x,data[4,:], label= "run 5")
plot(x,np.sum(np.abs(data), axis = 0)/data.shape[0], label="mean of {} runs".format(data.shape[0]))
legend(loc="best")
show()

In [8]:
# run simulations for <M>, <E> and <C> = <E^2> - <E>^2

TheWalker = randomWalker(40)

temp = np.array([10,4,3,2.5,2.27,2.2,2.1,2,1.5,1,0.5,0.1])
meanE = np.zeros(temp.size)
meanM = np.zeros(temp.size)
meanC = np.zeros(temp.size)

n_samples = 10
n_iter = 200
n_steps = 20

for i,T in enumerate(temp):
    Esquared = 0
    E = 0
    M = 0
    for n in range(n_samples):
        TheWalker.randomize()
        for it in range(n_iter):
            check = TheWalker.performMCS(T,n_steps)
            if (n_iter > 10):
                M += np.abs(TheWalker.getMagnetisation())
                thisE = TheWalker.getEnergy()
                E += thisE
                Esquared += (thisE*thisE)
            
    meanE[i] = E/(n_samples*n_iter)
    meanM[i] = M/(n_samples*n_iter)
    meanC[i] = (Esquared/(n_samples*n_samples) - E*E/(n_iter*n_iter) )

KeyboardInterrupt: 

In [ ]:
figure()

# subplot magnetisation
subplot(311)
plot(temp,meanM, label="magnetisation")
#plot(tempertatures,meanE/normalization, label="energy")
xlabel("T")
ylabel("M")
xlim(0.1,5)
legend(loc="best")
title("Ising Magnetisation Importance Sampling")

# subplot energy
subplot(312)
plot(temp,meanE, label="energy")
#plot(tempertatures,meanE/normalization, label="energy")
xlabel("T")
ylabel("E")
xlim(0.1,5)
legend(loc="best")
title("Ising Energy Importance Sampling")

# subplot susceptebility
subplot(313)
plot(temp,meanC, label="heat capacity")
#plot(tempertatures,meanE/normalization, label="energy")
xlabel("T")
ylabel("C")
xlim(0.1,5)
legend(loc="best")
title("Ising Heat Capacity Importance Sampling")

# adjust plot spacing
subplots_adjust(hspace=.3)

show()